# Question 3

Oilco has oil fields in Los Angeles and San Diego. The Los Angeles field can produce 400,000 barrels
per day and the San Diego field can produce 500,000 barrels per day.

Oil is sent from the fields to a refinery, in either Dallas or Houston (assume each refinery has unlimited
capacity). To refine 100,000 barrels costs $700 at Dallas and $900 at Houston.

Refined oil is shipped to customers in New York and Chicago. New York customers require 300,000
barrels per day and Chicago customers require 400,000 barrels per day.

The costs of shipping 100,000 barrels of oil (refined or unrefined) between cities are shown below.

|From/To($) | Dallas | Houston | New York | Chicago |
|:---------:|:------:|:-------:|:--------:|:-------:|
|Los Angels |300     |   110   |    -     |     -   |
|San Diego  |420     |   100   |    -     |    -    |
|Dallas     |-       |   -     |   450    |   550   |
|Houston    |-       |   -     |   470    | 530     |

In [1]:
using JuMP, HiGHS

# defining model
oilco = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [2]:
numedges = 12

#Define the incidence matrix A for our fully connected network
A = [1  1  1  0  0  0  0  0  0  0  0  0 
     0  0  0  1  1  1  0  0  0  0  0  0 
    -1  0  0 -1  0  0  1  1  1  0  0  0 
     0 -1  0  0 -1  0  0  0  0  1  1  1
     0  0  0  0  0  0 -1  0  0 -1  0  0
     0  0  0  0  0  0  0 -1  0  0 -1  0 
     0  0 -1  0  0 -1  0  0 -1  0  0 -1  
   ]

#Supply (first 2), relay (middle 2), demand (last 3)
b = [400000, 500000, 0, 0, -300000, -400000, -200000]

# costs of shipping per 100000 barrels
costs = [300 110 0 420 100 0 1150 1250 0 1370 1430 0];

A

7×12 Matrix{Int64}:
  1   1   1   0   0   0   0   0   0   0   0   0
  0   0   0   1   1   1   0   0   0   0   0   0
 -1   0   0  -1   0   0   1   1   1   0   0   0
  0  -1   0   0  -1   0   0   0   0   1   1   1
  0   0   0   0   0   0  -1   0   0  -1   0   0
  0   0   0   0   0   0   0  -1   0   0  -1   0
  0   0  -1   0   0  -1   0   0  -1   0   0  -1

In [3]:
@variable(oilco, x[1:numedges] >= 0)

@constraint(oilco, supplyanddemand, A*x .== b)

@objective(oilco, Min, sum(costs[i]*x[i] for i in 1:numedges))

print(oilco)

In [4]:
optimize!(oilco)

Presolving model
5 rows, 10 cols, 20 nonzeros
4 rows, 10 cols, 17 nonzeros
Presolve : Reductions: rows 4(-3); columns 10(-2); elements 17(-7)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     8.4500000000e+08 Pr: 3(1.4e+06) 0s
          5     1.0470000000e+09 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  1.0470000000e+09
HiGHS run time      :          0.00


In [5]:
value.(x)

12-element Vector{Float64}:
 300000.0
      0.0
 100000.0
      0.0
 400000.0
 100000.0
 300000.0
      0.0
      0.0
      0.0
 400000.0
      0.0